In [1]:
%run './init.py'

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train= mnist.train.images
y_train= mnist.train.labels

X_test= mnist.test.images
y_test= mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
n_steps = 500
display_step = 10
batch_size = 128

n_inputs = 784 
n_classes = 10 

X = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [4]:
weights = {
    # Weights of convolutional layers
    # Shape -> (filter width, filter height, no. channels, no. feature maps)
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    
    # Densely connected layer. We're using max pooling with kernel size of 2
    # after each convolutional layer so the size will be 4 ceil(28/2/2/2)
    'wd1': tf.Variable(tf.random_normal([4*4*64, 1024])),
    
    # The output layer (softmax)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}


# x will be in the form: (n_batches, height, width, n_channels)
# The default input format is NHWC
# strides[0] and strides[3] must default to 1
def conv2d(X, W, strides=1):
    X = tf.nn.conv2d(X, W, strides=[1, strides, strides, 1], padding='SAME')
    return tf.nn.relu(X)


def maxpool2d(X, k=2):
    return tf.nn.max_pool(X, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def conv_net(X, weights):
    # Reshape the input vector to 28x28 matrix
    X = tf.reshape(X, shape=[-1, 28, 28, 1])

    conv1 = conv2d(X, weights['wc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    conv3 = conv2d(conv2, weights['wc3'])
    conv3 = maxpool2d(conv3, k=2)

    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.matmul(fc1, weights['wd1'])
    fc1 = tf.nn.relu(fc1)

    out = tf.matmul(fc1, weights['out'])
    return out

In [5]:
pred = conv_net(X, weights)

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=y)
cost = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(cost)

correct = tf.nn.in_top_k(pred, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1, n_steps+1):
        batch_x, batch_y = random_batch(X_train, y_train, batch_size)
        sess.run(training_op, feed_dict={X: batch_x, y: batch_y})
        if step % display_step == 0 or step == 1:
            acc = sess.run( accuracy, feed_dict={X: batch_x, y: batch_y})
            print('Step:', step, 'Accuracy:', acc)
            
    test_acc = sess.run(accuracy, feed_dict={X: mnist.test.images[:100],y: mnist.test.labels[:100]})
    print("Test Accuracy:", test_acc)

Step: 1 Accuracy: 0.125
Step: 10 Accuracy: 0.203125
Step: 20 Accuracy: 0.34375
Step: 30 Accuracy: 0.515625
Step: 40 Accuracy: 0.59375
Step: 50 Accuracy: 0.734375
Step: 60 Accuracy: 0.632812
Step: 70 Accuracy: 0.796875
Step: 80 Accuracy: 0.734375
Step: 90 Accuracy: 0.757812
Step: 100 Accuracy: 0.78125
Step: 110 Accuracy: 0.8125
Step: 120 Accuracy: 0.828125
Step: 130 Accuracy: 0.867188
Step: 140 Accuracy: 0.867188
Step: 150 Accuracy: 0.851562
Step: 160 Accuracy: 0.828125
Step: 170 Accuracy: 0.851562
Step: 180 Accuracy: 0.890625
Step: 190 Accuracy: 0.875
Step: 200 Accuracy: 0.921875
Step: 210 Accuracy: 0.914062
Step: 220 Accuracy: 0.851562
Step: 230 Accuracy: 0.84375
Step: 240 Accuracy: 0.914062
Step: 250 Accuracy: 0.84375
Step: 260 Accuracy: 0.882812
Step: 270 Accuracy: 0.882812
Step: 280 Accuracy: 0.929688
Step: 290 Accuracy: 0.90625
Step: 300 Accuracy: 0.9375
Step: 310 Accuracy: 0.9375
Step: 320 Accuracy: 0.945312
Step: 330 Accuracy: 0.867188
Step: 340 Accuracy: 0.867188
Step: 350 Accu